In [416]:
import os
import sys
import numpy as np
import cPickle
import random
import math
import h5py
from multiprocessing import cpu_count
import scipy.io as sio
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from six import iteritems
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle as skshuffle
from numpy import genfromtxt
import itertools
import pickle


from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

np.random.seed(1)
random.seed(1)

## Evaluatoin

In [11]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

In [12]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [376]:
np.random.seed(1)
random.seed(1)
def scoring(emb_filename, matfile, mode = None, use_feature = False):
    # 0. Files
    embeddings_file = emb_filename

    if (mode == 'deepwalk-reproduce' or mode == 'semi' or mode == 'icml'):
        # 1. Load Embeddings
        embeddings = np.loadtxt(embeddings_file)
    elif (mode == 'deepwalk-original'):
        ## for original deepwalk
        model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    elif (mode == 'deepwalk-word2veclib'):
        ## for external word2vec lib
        model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')


    # 2. Load labels
    if (mode == 'icml'):
        NAMES = ['x', 'y', 'tx', 'ty', 'graph']
        OBJECTS = []
        print(DATASET)
        for i in range(len(NAMES)):
            f = file_path + 'trans.{}.{}'.format(DATASET, NAMES[i])
            OBJECTS.append(cPickle.load(open(f)))
        x, y, tx, ty, graph = tuple(OBJECTS)
        labels_matrix = np.concatenate([y, ty])
        labels_count = labels_matrix.shape[1]
        mlb = MultiLabelBinarizer(range(labels_count))
    else:
        mat = sio.loadmat(matfile)
        A = mat['network']
        graph = sparse2graph(A)
        labels_matrix = mat['group']
        labels_count = labels_matrix.shape[1]
        mlb = MultiLabelBinarizer(range(labels_count))

    if (mode == 'deepwalk-reproduce' or mode == 'semi' or mode == 'icml'):
        # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
        features_matrix = embeddings
    elif (mode == 'deepwalk-original'):
        # original code
        features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])
    elif (mode == 'deepwalk-word2veclib'):
        # use other word2vec lib
        features_matrix = np.asarray([model[str(node)] for node in range(len(model.vocab)-1)])
        rand_list = range(len(model.vocab)-1)
        features_matrix = np.asarray([model[str(node)] for node in rand_list])
        
    
    if (mode == 'icml'):
        all_results = defaultdict(list)
        if (use_feature):
            features_matrix = np.concatenate([x.toarray(), tx.toarray()])
            X, y = features_matrix, labels_matrix
        else:
            X, y = features_matrix, labels_matrix
        
        trn_x_id = range(0, x.shape[0])
        trn_y = y
        tst_x_id = range(x.shape[0], x.shape[0] + tx.shape[0])
        tst_y = tx
            
        X_train = X[trn_x_id, :]
        y_train = y[trn_x_id]

        X_test = X[tst_x_id, :]
        y_test = y[tst_x_id]

        clf = TopKRanker(LogisticRegression())
        clf.fit(X_train, y_train)

        # find out how many labels should be predicted
        top_k_list = [len(l) for l in y_test]
        preds = clf.predict(X_test, top_k_list)

        results = {}
        averages = ["micro", "macro"]
        for average in averages:
            results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)
        all_results['icml'].append(results)
    elif (mode == 'semi'):
        # read from trn and tst file
        ## Read the data, the trn_x is the position in the embedding matrix
        all_results = defaultdict(list)
        X, y = features_matrix, labels_matrix
        

        with open(data_splited_filename, 'rb') as handle:
            unserialized_data = pickle.load(handle)
        trn_x_id, trn_y, tst_x_id, tst_y = (unserialized_data['trn_x'], unserialized_data['trn_y'],\
                                            unserialized_data['tst_x'], unserialized_data['tst_y'])
            
        X_train = X[trn_x_id, :]
        y_train_ = y[trn_x_id]

        y_train = [[] for x in range(y_train_.shape[0])]
        
        cy =  y_train_.tocoo()
        for i, j in zip(cy.row, cy.col):
            y_train[i].append(j)

        assert sum(len(l) for l in y_train) == y_train_.nnz

        X_test = X[tst_x_id, :]
        y_test_ = y[tst_x_id]

        y_test = [[] for _ in range(y_test_.shape[0])]

        cy =  y_test_.tocoo()
        for i, j in zip(cy.row, cy.col):
            y_test[i].append(j)

        clf = TopKRanker(LogisticRegression())
        clf.fit(X_train, y_train_)

        # find out how many labels should be predicted
        top_k_list = [len(l) for l in y_test]
        preds = clf.predict(X_test, top_k_list)

        results = {}
        averages = ["micro", "macro"]
        for average in averages:
            results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)
#             print('=============y_pred==========')
#             print(preds)
#             print('=============y_test==========')
#             print(y_test)
        all_results['semi'].append(results)
    else:
        # 2. Shuffle, to create train/test groups
        shuffles = []
        for x in range(1):
            shuffles.append(skshuffle(features_matrix, labels_matrix, random_state = 1))

        # 3. to score each train/test group
        all_results = defaultdict(list)

    #     if args.all:
    #         training_percents = numpy.asarray(range(1, 10)) * .1
    #     else:
    #         training_percents = [0.1, 0.5, 0.9]
        training_percents = [0.1]
        for train_percent in training_percents:
            for shuf in shuffles:
                
                X, y = shuf

                training_size = int(train_percent * X.shape[0])

                X_train = X[:training_size, :]
                y_train_ = y[:training_size]

                y_train = [[] for x in range(y_train_.shape[0])]


                cy =  y_train_.tocoo()
                for i, j in zip(cy.row, cy.col):
                    y_train[i].append(j)

                assert sum(len(l) for l in y_train) == y_train_.nnz

                X_test = X[training_size:, :]
                y_test_ = y[training_size:]

                y_test = [[] for _ in range(y_test_.shape[0])]

                cy =  y_test_.tocoo()
                for i, j in zip(cy.row, cy.col):
                    y_test[i].append(j)

                clf = TopKRanker(LogisticRegression())
                clf.fit(X_train, y_train_)

                # find out how many labels should be predicted
                top_k_list = [len(l) for l in y_test]
                preds = clf.predict(X_test, top_k_list)

                results = {}
                averages = ["micro", "macro"]
                for average in averages:
                    results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)
                all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')

In [378]:
DATASET = 'citeseer'
matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
embedding_filename = '/hdd2/graph_embedding/customized/results/exp_citeseer_unsupervised/citeseer_embeddings_iter0.txt'
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
embedding_filename_other_lib = '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin'

data_splited_filename = '/hdd2/graph_embedding/customized/blogcatalog_splited_p90.pickle'

scoring(embedding_filename, matfile, 'icml', use_feature = False)

citeseer
('Results, using embeddings of dimensionality', 128)
-------------------
('Train percent:', 'icml')
('Shuffle #1:   ', {'micro': 0.5, 'macro': 0.33333333333333331})
('Average score:', {'micro': 0.5, 'macro': 0.33333333333333331})
-------------------


In [23]:
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
model = KeyedVectors.load_word2vec_format(embedding_filename_original, binary=False)
                                          
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embeddings = np.loadtxt(embedding_filename)


## Generate Data

In [124]:
# def write_walks_to_disk(G, filebase, num_paths, path_length, alpha=0, rand=random.Random(0), num_workers=cpu_count(),
#                         always_rebuild=True):
#     global __current_graph
#     __current_graph = G
#     files_list = ["{}.{}".format(filebase, str(x)) for x in list(range(num_paths))]
#     expected_size = len(G)
#     args_list = []
#     files = []

#     if num_paths <= num_workers:
#         paths_per_worker = [1 for x in range(num_paths)]
#     else:
#         paths_per_worker = [len(list(filter(lambda z: z!= None, [y for y in x])))
#                             for x in graph.grouper(int(num_paths / num_workers)+1, range(1, num_paths+1))]

#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         for size, file_, ppw in zip(executor.map(count_lines, files_list), files_list, paths_per_worker):
#             if always_rebuild or size != (ppw*expected_size):
#                 args_list.append((ppw, path_length, alpha, random.Random(rand.randint(0, 2**31)), file_))
#             else:
#                 files.append(file_)

#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         for file_ in executor.map(_write_walks_to_disk, args_list):
#             files.append(file_)

#     return files



def random_walk(graph, num_paths, path_length, rand = random.Random(0)):
    vocabulary_size = max(graph.keys()) + 1
    i = 0
    list_path = []
    tot_path = 0
    nodes = range(vocabulary_size)
    for cnt in range(num_paths):
        rand.shuffle(nodes)
        for node in nodes:
            path = [node]
            for _ in xrange(path_length-1):
                path.append(rand.choice(graph[path[-1]]))
            list_path.append(path)
    return list_path
    

In [125]:
## adapt the icml paper to deepwalk
DATASET = 'citeseer'
number_walks = 50
walk_length = 40
seed = 1 
workers = 1
output_filename = '/hdd2/graph_embedding/customized/tmp/%s.embeddings' %(DATASET)
def generate_data_icml(file_path, DATASET):
    NAMES = ['x', 'y', 'tx', 'ty', 'graph']
    OBJECTS = []
    for i in range(len(NAMES)):
        f = file_path + 'trans.{}.{}'.format(DATASET, NAMES[i])
        print(f)
        OBJECTS.append(cPickle.load(open(f)))
    x, y, tx, ty, graph = tuple(OBJECTS)
    
    num_walks = len(graph) * number_walks
    
    print("Walking...")
    walk = random_walk(graph, number_walks, walk_length)

    return walk
file_path = '/hdd2/graph_embedding/planetoid/data/'
walks_filebase = output_filename + ".walks"
walk = generate_data_icml(file_path, DATASET)
np.savetxt(walks_filebase, walk, delimiter=' ')

/hdd2/graph_embedding/planetoid/data/trans.citeseer.x
/hdd2/graph_embedding/planetoid/data/trans.citeseer.y
/hdd2/graph_embedding/planetoid/data/trans.citeseer.tx
/hdd2/graph_embedding/planetoid/data/trans.citeseer.ty
/hdd2/graph_embedding/planetoid/data/trans.citeseer.graph
Walking...


In [380]:
DATASET = 'citeseer'
NAMES = ['x', 'y', 'tx', 'ty', 'graph']
OBJECTS = []
for i in range(len(NAMES)):
    f = file_path + 'trans.{}.{}'.format(DATASET, NAMES[i])
    print(f)
    OBJECTS.append(cPickle.load(open(f)))
x, y, tx, ty, graph = tuple(OBJECTS)

/hdd2/graph_embedding/planetoid/data/trans.citeseer.x
/hdd2/graph_embedding/planetoid/data/trans.citeseer.y
/hdd2/graph_embedding/planetoid/data/trans.citeseer.tx
/hdd2/graph_embedding/planetoid/data/trans.citeseer.ty
/hdd2/graph_embedding/planetoid/data/trans.citeseer.graph


In [417]:
x.shape

(120, 3703)

In [382]:
from sklearn import linear_model, datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

logreg = LogisticRegression()
fx = x.toarray()
y_true = [np.nonzero(e)[0][0] for e in y]
ftx = tx.toarray()
ty_true = [np.nonzero(e)[0][0] for e in ty]

In [446]:
embedding_filename = '/hdd2/graph_embedding/customized/results/exp_citeseer_semi/citeseer_embeddings_iter26.txt'
embeddings = np.loadtxt(embedding_filename)
emb_x = embeddings[:x.shape[0] , :]
emb_tx = embeddings[x.shape[0]:x.shape[0] + ftx.shape[0], :]

xx = np.concatenate([fx, emb_x], axis = 1)
txx = np.concatenate([ftx, emb_tx], axis = 1)
# xx = fx
# txx = ftx

# xx = emb_x
# txx = emb_tx

In [447]:
logreg.fit(xx, y_true)
ty_pred = logreg.predict(txx)
print('acc: %.8f' %(accuracy_score(ty_true, ty_pred)))
print('macro f1: %.8f' %f1_score(ty_true, ty_pred, average='macro'))
print('micro f1: %.8f' %f1_score(ty_true, ty_pred, average='micro'))

acc: 0.65800000
macro f1: 0.62749644
micro f1: 0.65800000


In [390]:
xx.shape

(120, 128)

In [281]:
tx.shape

(120, 3831)

In [415]:
x[[1,2]]

<2x3703 sparse matrix of type '<type 'numpy.float32'>'
	with 58 stored elements in Compressed Sparse Row format>